In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.metrics import confusion_matrix, classification_report

# Importing the models

import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# Importing sampling methods

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks

!pip install optuna
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

TRIALS = 5
MODEL_NUM = 5
NUM_ITER = 1000
SEED = 2024
SEED_OBJ = SEED
SAMPLER = TPESampler(seed=SEED)
max_metrics = []
max_val_auc_score = 0
avg_accuracy, avg_auc, avg_f1, avg_precision, avg_recall = 0, 0, 0, 0, 0

kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)

import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
htru2_data = pd.read_csv('https://raw.githubusercontent.com/szbela87/ml_22_elteik/main/data/HTRU_2.csv', header=None)
htru2_data.columns = ['mean_ip', 'std_ip', 'excess_kurt_ip', 'skewness_ip', 'mean_DMSNR', 'std_DMSNR', 'excess_kurt_DMSNR', 'skewness_DMSNR', 'class']

In [3]:
htru2_data

,mean_ip,std_ip,excess_kurt_ip,skewness_ip,mean_DMSNR,std_DMSNR,excess_kurt_DMSNR,skewness_DMSNR,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0


In [4]:
htru2_data.describe()

,mean_ip,std_ip,excess_kurt_ip,skewness_ip,mean_DMSNR,std_DMSNR,excess_kurt_DMSNR,skewness_DMSNR,class
count,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,111.079968,46.549532,0.477857,1.770279,12.614400,26.326515,8.303556,104.857709,0.091574
std,25.652935,6.843189,1.064040,6.167913,29.472897,19.470572,4.506092,106.514540,0.288432
min,5.812500,24.772042,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.929688,42.376018,0.027098,-0.188572,1.923077,14.437332,5.781506,34.960504,0.000000
50%,115.078125,46.947479,0.223240,0.198710,2.801839,18.461316,8.433515,83.064556,0.000000
75%,127.085938,51.023202,0.473325,0.927783,5.464256,28.428104,10.702959,139.309330,0.000000
max,192.617188,98.778911,8.069522,68.101622,223.392141,110.642211,34.539844,1191.000837,1.000000


In [5]:
htru2_data.shape

(17898, 9)

In [6]:
htru2_data["class"].value_counts()

class
0    16259
1     1639
Name: count, dtype: int64

In [7]:
# Splitting the data (80/10/10% - train/validation/test)

X = htru2_data.drop("class", axis=1)
y = htru2_data["class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((16108, 8), (16108,), (1790, 8), (1790,))

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1/0.9, random_state=SEED)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((14318, 8), (14318,), (1790, 8), (1790,))

In [9]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((14318, 8), (14318,), (1790, 8), (1790,), (1790, 8), (1790,))

In [10]:
y_train.value_counts(), y_val.value_counts(), y_test.value_counts()

(class
 0    13014
 1     1304
 Name: count, dtype: int64,
 class
 0    1627
 1     163
 Name: count, dtype: int64,
 class
 0    1618
 1     172
 Name: count, dtype: int64)

# Baseline XGBoost

In [11]:
xgb_clf = xgb.XGBClassifier(random_state=SEED)
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall score: {recall:.4f}")
print(f"Confusion matrix:\n {conf_matrix}")

Accuracy: 0.9816
AUC: 0.9430
F1-score: 0.9032
Precision: 0.9112
Recall score: 0.8953
Confusion matrix:
 [[1603   15]
 [  18  154]]


# Baseline XGBoost fine tuning with Optuna

In [12]:
for i in range(TRIALS):

  t1 = time.perf_counter()
  SEED = SEED + 1

  def objective(trial):
    params = {
        "learning_rate" : trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "max_depth" : trial.suggest_int("max_depth", 3, 10),
        "gamma" : trial.suggest_uniform("gamma", 0, 0.5),
        "colsample_bytree" : trial.suggest_uniform("colsample_bytree", 0.5, 1),
        "reg_alpha" : trial.suggest_loguniform("reg_alpha", 1.0e-8, 1),
        "reg_lambda" : trial.suggest_loguniform("reg_lambda", 1.0e-8, 10),
        "subsample" : trial.suggest_uniform("subsample", 0.5, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 50, 500)
    }

    xgb_clf = xgb.XGBClassifier(random_state=SEED, **params)

    # Fitting the model and evaluating with auc metric

    xgb_clf.fit(X_train, y_train)
    y_pred = xgb_clf.predict(X_val)

    auc_score = roc_auc_score(y_val, y_pred)

    return auc_score

  # Optuna sampler

  SAMPLER = TPESampler(seed=SEED)

  # Optuna study

  xgb_study = optuna.create_study(direction="maximize", sampler=SAMPLER)
  xgb_study.optimize(objective, n_trials=NUM_ITER)

  # Optuna best parameters

  best_params = xgb_study.best_trial.params

  # Fine tuned XGBoost

  xgb_clf_optuna = xgb.XGBClassifier(random_state=SEED, **best_params)
  xgb_clf_optuna.fit(X_train, y_train)

  y_pred = xgb_clf_optuna.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)

  # Max AUC validation scores

  if max_val_auc_score < xgb_study.best_trial.value:
    max_val_auc_score = xgb_study.best_trial.value
    max_metrics = [accuracy, auc, f1, precision, recall]

  # Average scores

  avg_accuracy += accuracy
  avg_auc += auc
  avg_f1 += f1
  avg_precision += precision
  avg_recall += recall

  print("\n" + 10*"/" + "*")
  print(f"Trial {i+1}" + 3*" " + "*")
  print(10*"/" + "*")

  print(f"\nAccuracy: {accuracy:.4f}")
  print(f"AUC: {auc:.4f}")
  print(f"F1-score: {f1:.4f}")
  print(f"Precision: {precision:.4f}")
  print(f"Recall score: {recall:.4f}")
  print(f"Confusion matrix:\n {conf_matrix}")

  t2 = time.perf_counter()
  print("Runtime:", t2-t1)

# Best validation auc scores

print("\n" + 50*"-")
print("\n" + 42*"/" + "*")
print("Scores with the best validation AUC score *")
print(42*"/" + "*")

print(f"\nBest validation AUC: {(max_val_auc_score):.4f}")
print(f"Accuracy: {(max_metrics[0]):.4f}")
print(f"AUC: {(max_metrics[1]):.4f}")
print(f"F1-score: {(max_metrics[2]):.4f}")
print(f"Precision: {(max_metrics[3]):.4f}")
print(f"Recall score: {(max_metrics[4]):.4f}")

# Printing average scores

print("\n" + 50*"-")
print("\n" + 15*"/" + "*")
print("Average scores *")
print(15*"/" + "*")

print(f"\nAverage accuracy: {(avg_accuracy / TRIALS):.4f}")
print(f"Average AUC: {(avg_auc / TRIALS):.4f}")
print(f"Average f1-score: {(avg_f1 / TRIALS):.4f}")
print(f"Average precision: {(avg_precision / TRIALS):.4f}")
print(f"Average recall score: {(avg_recall / TRIALS):.4f}")

# Saving the model

pickle.dump(xgb_clf_optuna, open("baseline_xgboost_without_smote.pkl","wb"))


//////////*
Trial 1   *
//////////*

Accuracy: 0.9810
AUC: 0.9401
F1-score: 0.9000
Precision: 0.9107
Recall score: 0.8895
Confusion matrix:
 [[1603   15]
 [  19  153]]
Runtime: 439.80205387668684

//////////*
Trial 2   *
//////////*

Accuracy: 0.9832
AUC: 0.9414
F1-score: 0.9107
Precision: 0.9329
Recall score: 0.8895
Confusion matrix:
 [[1607   11]
 [  19  153]]
Runtime: 601.1243723891675

//////////*
Trial 3   *
//////////*

Accuracy: 0.9821
AUC: 0.9408
F1-score: 0.9053
Precision: 0.9217
Recall score: 0.8895
Confusion matrix:
 [[1605   13]
 [  19  153]]
Runtime: 481.3440371719189

//////////*
Trial 4   *
//////////*

Accuracy: 0.9821
AUC: 0.9382
F1-score: 0.9048
Precision: 0.9268
Recall score: 0.8837
Confusion matrix:
 [[1606   12]
 [  20  152]]
Runtime: 390.9174131369218

//////////*
Trial 5   *
//////////*

Accuracy: 0.9838
AUC: 0.9417
F1-score: 0.9134
Precision: 0.9387
Recall score: 0.8895
Confusion matrix:
 [[1608   10]
 [  19  153]]
Runtime: 304.3367693917826

------------------

# Baseline XGBoost with SMOTE and fine tuning with Optuna

In [13]:
max_metrics = []
max_val_auc_score = 0
avg_accuracy, avg_auc, avg_f1, avg_precision, avg_recall = 0, 0, 0, 0, 0

for i in range(TRIALS):

  t1 = time.perf_counter()
  SEED = SEED + 1

  def objective(trial):

    # XGBoost parameters

    params = {
        "learning_rate" : trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "max_depth" : trial.suggest_int("max_depth", 3, 10),
        "gamma" : trial.suggest_uniform("gamma", 0, 0.5),
        "colsample_bytree" : trial.suggest_uniform("colsample_bytree", 0.5, 1),
        "reg_alpha" : trial.suggest_loguniform("reg_alpha", 1e-8, 1),
        "reg_lambda" : trial.suggest_loguniform("reg_lambda", 1e-8, 10),
        "subsample" : trial.suggest_uniform("subsample", 0.5, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 50, 500)
    }

    xgb_clf = xgb.XGBClassifier(random_state=SEED, **params)

    # SMOTE parameter

    smote_kn = trial.suggest_int("k_neighbors", 5, 50)

    smote_train = SMOTE(sampling_strategy="minority", k_neighbors=smote_kn, random_state=SEED)
    X_smote_train, y_smote_train = smote_train.fit_resample(X_train, y_train)

    # Fitting the model and evaluating with auc metric

    xgb_clf.fit(X_smote_train, y_smote_train)
    y_pred = xgb_clf.predict(X_val)

    auc_score = roc_auc_score(y_val, y_pred)

    return auc_score

  # Optuna sampler

  SAMPLER = TPESampler(seed=SEED)

  # Optuna study

  xgb_study = optuna.create_study(direction="maximize", sampler=SAMPLER)
  xgb_study.optimize(objective, n_trials=NUM_ITER)

  # Optuna best parameters

  best_params = xgb_study.best_trial.params

  # SMOTE best parameter

  smote_kn = best_params["k_neighbors"]

  smote_train = SMOTE(sampling_strategy="minority", k_neighbors=smote_kn, random_state=SEED)
  X_smote_train, y_smote_train = smote_train.fit_resample(X_train, y_train)

  del best_params["k_neighbors"]

  # Fine tuned XGBoost

  xgb_clf_optuna = xgb.XGBClassifier(random_state=SEED, **best_params)
  xgb_clf_optuna.fit(X_smote_train, y_smote_train)

  y_pred = xgb_clf_optuna.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)

  # Max AUC validation scores

  if max_val_auc_score < xgb_study.best_trial.value:
    max_val_auc_score = xgb_study.best_trial.value
    max_metrics = [accuracy, auc, f1, precision, recall]

  # Average scores

  avg_accuracy += accuracy
  avg_auc += auc
  avg_f1 += f1
  avg_precision += precision
  avg_recall += recall

  print("\n" + 10*"/" + "*")
  print(f"Trial {i+1}" + 3*" " + "*")
  print(10*"/" + "*")

  print(f"\nAccuracy: {accuracy:.4f}")
  print(f"AUC: {auc:.4f}")
  print(f"F1-score: {f1:.4f}")
  print(f"Precision: {precision:.4f}")
  print(f"Recall score: {recall:.4f}")
  print(f"Confusion matrix:\n {conf_matrix}")

  t2 = time.perf_counter()
  print("Runtime:", t2-t1)

# Best validation auc scores

print("\n" + 50*"-")
print("\n" + 42*"/" + "*")
print("Scores with the best validation AUC score *")
print(42*"/" + "*")

print(f"\nBest validation AUC: {(max_val_auc_score):.4f}")
print(f"Accuracy: {(max_metrics[0]):.4f}")
print(f"AUC: {(max_metrics[1]):.4f}")
print(f"F1-score: {(max_metrics[2]):.4f}")
print(f"Precision: {(max_metrics[3]):.4f}")
print(f"Recall score: {(max_metrics[4]):.4f}")

# Printing average scores

print("\n" + 50*"-")
print("\n" + 15*"/" + "*")
print("Average scores *")
print(15*"/" + "*")

print(f"\nAverage accuracy: {(avg_accuracy / TRIALS):.4f}")
print(f"Average AUC: {(avg_auc / TRIALS):.4f}")
print(f"Average f1-score: {(avg_f1 / TRIALS):.4f}")
print(f"Average precision: {(avg_precision / TRIALS):.4f}")
print(f"Average recall score: {(avg_recall / TRIALS):.4f}")

# Saving the model

pickle.dump(xgb_clf_optuna, open("baseline_xgboost_with_smote.pkl","wb"))


//////////*
Trial 1   *
//////////*

Accuracy: 0.9704
AUC: 0.9473
F1-score: 0.8564
Precision: 0.8020
Recall score: 0.9186
Confusion matrix:
 [[1579   39]
 [  14  158]]
Runtime: 650.7857317430899

//////////*
Trial 2   *
//////////*

Accuracy: 0.9721
AUC: 0.9508
F1-score: 0.8641
Precision: 0.8112
Recall score: 0.9244
Confusion matrix:
 [[1581   37]
 [  13  159]]
Runtime: 565.7073269141838

//////////*
Trial 3   *
//////////*

Accuracy: 0.9682
AUC: 0.9460
F1-score: 0.8472
Precision: 0.7861
Recall score: 0.9186
Confusion matrix:
 [[1575   43]
 [  14  158]]
Runtime: 421.06087436620146

//////////*
Trial 4   *
//////////*

Accuracy: 0.9726
AUC: 0.9459
F1-score: 0.8650
Precision: 0.8220
Recall score: 0.9128
Confusion matrix:
 [[1584   34]
 [  15  157]]
Runtime: 563.9369056499563

//////////*
Trial 5   *
//////////*

Accuracy: 0.9726
AUC: 0.9459
F1-score: 0.8650
Precision: 0.8220
Recall score: 0.9128
Confusion matrix:
 [[1584   34]
 [  15  157]]
Runtime: 738.0342572377995

------------------

# Baseline XGBoost with Oversampling (ADASYN) and fine tuning with Optuna

In [14]:
max_metrics = []
max_val_auc_score = 0
avg_accuracy, avg_auc, avg_f1, avg_precision, avg_recall = 0, 0, 0, 0, 0

for i in range(TRIALS):

  t1 = time.perf_counter()
  SEED = SEED + 1

  def objective(trial):

    # XGBoost parameters

    params = {
        "learning_rate" : trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "max_depth" : trial.suggest_int("max_depth", 3, 10),
        "gamma" : trial.suggest_uniform("gamma", 0, 0.5),
        "colsample_bytree" : trial.suggest_uniform("colsample_bytree", 0.5, 1),
        "reg_alpha" : trial.suggest_loguniform("reg_alpha", 1e-8, 1),
        "reg_lambda" : trial.suggest_loguniform("reg_lambda", 1e-8, 10),
        "subsample" : trial.suggest_uniform("subsample", 0.5, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 50, 500)
    }

    xgb_clf = xgb.XGBClassifier(random_state=SEED, **params)

    # ADASYN parameter

    adasyn_nn = trial.suggest_int("n_neighbors", 5, 50)

    adasyn_train = ADASYN(sampling_strategy="minority", n_neighbors=adasyn_nn, random_state=SEED_OBJ)
    X_adasyn_train, y_adasyn_train = adasyn_train.fit_resample(X_train, y_train)

    # Fitting the model and evaluating with auc metric

    xgb_clf.fit(X_adasyn_train, y_adasyn_train)
    y_pred = xgb_clf.predict(X_val)

    auc_score = roc_auc_score(y_val, y_pred)

    return auc_score

  # Optuna sampler

  SAMPLER = TPESampler(seed=SEED)

  # Optuna study

  xgb_study = optuna.create_study(direction="maximize", sampler=SAMPLER)
  xgb_study.optimize(objective, n_trials=NUM_ITER)

  # Optuna best parameters

  best_params = xgb_study.best_trial.params

  # ADASYN best parameter

  adasyn_nn = best_params["n_neighbors"]

  adasyn_train = ADASYN(sampling_strategy="minority", n_neighbors=adasyn_nn, random_state=SEED)
  X_adasyn_train, y_adasyn_train = adasyn_train.fit_resample(X_train, y_train)

  del best_params["n_neighbors"]

  # Fine tuned XGBoost

  xgb_clf_optuna = xgb.XGBClassifier(random_state=SEED, **best_params)
  xgb_clf_optuna.fit(X_adasyn_train, y_adasyn_train)

  y_pred = xgb_clf_optuna.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)

  # Max AUC validation scores

  if max_val_auc_score < xgb_study.best_trial.value:
    max_val_auc_score = xgb_study.best_trial.value
    max_metrics = [accuracy, auc, f1, precision, recall]

  # Average scores

  avg_accuracy += accuracy
  avg_auc += auc
  avg_f1 += f1
  avg_precision += precision
  avg_recall += recall

  print("\n" + 10*"/" + "*")
  print(f"Trial {i+1}" + 3*" " + "*")
  print(10*"/" + "*")

  print(f"\nAccuracy: {accuracy:.4f}")
  print(f"AUC: {auc:.4f}")
  print(f"F1-score: {f1:.4f}")
  print(f"Precision: {precision:.4f}")
  print(f"Recall score: {recall:.4f}")
  print(f"Confusion matrix:\n {conf_matrix}")

  t2 = time.perf_counter()
  print("Runtime:", t2-t1)

# Best validation auc scores

print("\n" + 50*"-")
print("\n" + 42*"/" + "*")
print("Scores with the best validation AUC score *")
print(42*"/" + "*")

print(f"\nBest validation AUC: {(max_val_auc_score):.4f}")
print(f"Accuracy: {(max_metrics[0]):.4f}")
print(f"AUC: {(max_metrics[1]):.4f}")
print(f"F1-score: {(max_metrics[2]):.4f}")
print(f"Precision: {(max_metrics[3]):.4f}")
print(f"Recall score: {(max_metrics[4]):.4f}")

# Printing average scores

print("\n" + 50*"-")
print("\n" + 15*"/" + "*")
print("Average scores *")
print(15*"/" + "*")

print(f"\nAverage accuracy: {(avg_accuracy / TRIALS):.4f}")
print(f"Average AUC: {(avg_auc / TRIALS):.4f}")
print(f"Average f1-score: {(avg_f1 / TRIALS):.4f}")
print(f"Average precision: {(avg_precision / TRIALS):.4f}")
print(f"Average recall score: {(avg_recall / TRIALS):.4f}")

# Saving the model

pickle.dump(xgb_clf_optuna, open("baseline_xgboost_with_adasyn.pkl","wb"))


//////////*
Trial 1   *
//////////*

Accuracy: 0.9503
AUC: 0.9361
F1-score: 0.7802
Precision: 0.6781
Recall score: 0.9186
Confusion matrix:
 [[1543   75]
 [  14  158]]
Runtime: 729.1517240679823

//////////*
Trial 2   *
//////////*

Accuracy: 0.9486
AUC: 0.9352
F1-score: 0.7745
Precision: 0.6695
Recall score: 0.9186
Confusion matrix:
 [[1540   78]
 [  14  158]]
Runtime: 820.6972627611831

//////////*
Trial 3   *
//////////*

Accuracy: 0.9508
AUC: 0.9338
F1-score: 0.7811
Precision: 0.6826
Recall score: 0.9128
Confusion matrix:
 [[1545   73]
 [  15  157]]
Runtime: 790.8358994391747

//////////*
Trial 4   *
//////////*

Accuracy: 0.9441
AUC: 0.9301
F1-score: 0.7585
Precision: 0.6488
Recall score: 0.9128
Confusion matrix:
 [[1533   85]
 [  15  157]]
Runtime: 866.496313364245

//////////*
Trial 5   *
//////////*

Accuracy: 0.9520
AUC: 0.9371
F1-score: 0.7861
Precision: 0.6870
Recall score: 0.9186
Confusion matrix:
 [[1546   72]
 [  14  158]]
Runtime: 873.3107528500259

--------------------

# Baseline XGBoost with Undersampling (TomekLinks) and fine tuning with Optuna

In [15]:
max_metrics = []
max_val_auc_score = 0
avg_accuracy, avg_auc, avg_f1, avg_precision, avg_recall = 0, 0, 0, 0, 0

for i in range(TRIALS):

  t1 = time.perf_counter()
  SEED = SEED + 1

  def objective(trial):

    # XGBoost parameters

    params = {
        "learning_rate" : trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "max_depth" : trial.suggest_int("max_depth", 3, 10),
        "gamma" : trial.suggest_uniform("gamma", 0, 0.5),
        "colsample_bytree" : trial.suggest_uniform("colsample_bytree", 0.5, 1),
        "reg_alpha" : trial.suggest_loguniform("reg_alpha", 1e-8, 1),
        "reg_lambda" : trial.suggest_loguniform("reg_lambda", 1e-8, 10),
        "subsample" : trial.suggest_uniform("subsample", 0.5, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 50, 500)
    }

    xgb_clf = xgb.XGBClassifier(random_state=SEED, **params)

    # TomekLinks

    tomek_train = TomekLinks(sampling_strategy="majority")
    X_tomek_train, y_tomek_train = tomek_train.fit_resample(X_train, y_train)

    # Fitting the model and evaluating with auc metric

    xgb_clf.fit(X_tomek_train, y_tomek_train)
    y_pred = xgb_clf.predict(X_val)

    auc_score = roc_auc_score(y_val, y_pred)

    return auc_score

  # Optuna sampler

  SAMPLER = TPESampler(seed=SEED)

  # Optuna study

  xgb_study = optuna.create_study(direction="maximize", sampler=SAMPLER)
  xgb_study.optimize(objective, n_trials=NUM_ITER)

  # Optuna best parameters

  best_params = xgb_study.best_trial.params

  # TomekLinks

  tomek_train = TomekLinks(sampling_strategy="majority")
  X_tomek_train, y_tomek_train = tomek_train.fit_resample(X_train, y_train)

  # Fine tuned XGBoost

  xgb_clf_optuna = xgb.XGBClassifier(random_state=SEED, **best_params)
  xgb_clf_optuna.fit(X_tomek_train, y_tomek_train)

  y_pred = xgb_clf_optuna.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)

  # Max AUC validation scores

  if max_val_auc_score < xgb_study.best_trial.value:
    max_val_auc_score = xgb_study.best_trial.value
    max_metrics = [accuracy, auc, f1, precision, recall]

  # Average scores

  avg_accuracy += accuracy
  avg_auc += auc
  avg_f1 += f1
  avg_precision += precision
  avg_recall += recall

  print("\n" + 10*"/" + "*")
  print(f"Trial {i+1}" + 3*" " + "*")
  print(10*"/" + "*")

  print(f"\nAccuracy: {accuracy:.4f}")
  print(f"AUC: {auc:.4f}")
  print(f"F1-score: {f1:.4f}")
  print(f"Precision: {precision:.4f}")
  print(f"Recall score: {recall:.4f}")
  print(f"Confusion matrix:\n {conf_matrix}")

  t2 = time.perf_counter()
  print("Runtime:", t2-t1)

# Best validation auc scores

print("\n" + 50*"-")
print("\n" + 42*"/" + "*")
print("Scores with the best validation AUC score *")
print(42*"/" + "*")

print(f"\nBest validation AUC: {(max_val_auc_score):.4f}")
print(f"Accuracy: {(max_metrics[0]):.4f}")
print(f"AUC: {(max_metrics[1]):.4f}")
print(f"F1-score: {(max_metrics[2]):.4f}")
print(f"Precision: {(max_metrics[3]):.4f}")
print(f"Recall score: {(max_metrics[4]):.4f}")

# Printing average scores

print("\n" + 50*"-")
print("\n" + 15*"/" + "*")
print("Average scores *")
print(15*"/" + "*")

print(f"\nAverage accuracy: {(avg_accuracy / TRIALS):.4f}")
print(f"Average AUC: {(avg_auc / TRIALS):.4f}")
print(f"Average f1-score: {(avg_f1 / TRIALS):.4f}")
print(f"Average precision: {(avg_precision / TRIALS):.4f}")
print(f"Average recall score: {(avg_recall / TRIALS):.4f}")

# Saving the model

pickle.dump(xgb_clf_optuna, open("baseline_xgboost_with_tomeklinks.pkl","wb"))


//////////*
Trial 1   *
//////////*

Accuracy: 0.9821
AUC: 0.9408
F1-score: 0.9053
Precision: 0.9217
Recall score: 0.8895
Confusion matrix:
 [[1605   13]
 [  19  153]]
Runtime: 508.28042079368606

//////////*
Trial 2   *
//////////*

Accuracy: 0.9827
AUC: 0.9411
F1-score: 0.9080
Precision: 0.9273
Recall score: 0.8895
Confusion matrix:
 [[1606   12]
 [  19  153]]
Runtime: 761.7633559540845

//////////*
Trial 3   *
//////////*

Accuracy: 0.9821
AUC: 0.9433
F1-score: 0.9059
Precision: 0.9167
Recall score: 0.8953
Confusion matrix:
 [[1604   14]
 [  18  154]]
Runtime: 630.5276851668023

//////////*
Trial 4   *
//////////*

Accuracy: 0.9810
AUC: 0.9401
F1-score: 0.9000
Precision: 0.9107
Recall score: 0.8895
Confusion matrix:
 [[1603   15]
 [  19  153]]
Runtime: 501.67964629409835

//////////*
Trial 5   *
//////////*

Accuracy: 0.9810
AUC: 0.9401
F1-score: 0.9000
Precision: 0.9107
Recall score: 0.8895
Confusion matrix:
 [[1603   15]
 [  19  153]]
Runtime: 557.7425364470109

-----------------